In [2]:
import os
os.chdir("../")

In [39]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import roc_auc_score

In [35]:
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
import catboost as cb
from sklearn.pipeline import Pipeline

# Data Set

In [44]:
X_train = pd.read_csv('data/preprocessed/X_train_scaled.csv')
X_test = pd.read_csv('data/preprocessed/X_test_scaled.csv')
y_train = pd.read_csv('data/preprocessed/y_train.csv')
y_test = pd.read_csv('data/preprocessed/y_test.csv')

X_submit = pd.read_csv('data/preprocessed/X_submission_scaled.csv')

In [45]:
X = pd.concat([X_train, X_test], ignore_index=True)
y = pd.concat([y_train, y_test], ignore_index=True)

# Gradient Boosting Decision Tree

## xgboost

In [11]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.1,
    'gamma': 0.0,
    'alpha': 0.0,
    'lambda': 1.0,
    'min_child_weight': 1,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

num_round = 100

evals = [(dtrain, 'train'), (dtest, 'eval')]

model = xgb.train(params, dtrain, num_round, evals=evals, early_stopping_rounds=10)

y_pred = model.predict(dtest)

auc_score = roc_auc_score(y_test, y_pred)
print(f"AUC Score: {auc_score:.4f}")

[0]	train-logloss:0.40172	eval-logloss:0.40389
[1]	train-logloss:0.38670	eval-logloss:0.39103


[2]	train-logloss:0.37444	eval-logloss:0.38002
[3]	train-logloss:0.36309	eval-logloss:0.37025
[4]	train-logloss:0.35361	eval-logloss:0.36369
[5]	train-logloss:0.34633	eval-logloss:0.35709
[6]	train-logloss:0.33854	eval-logloss:0.34948
[7]	train-logloss:0.33177	eval-logloss:0.34475
[8]	train-logloss:0.32521	eval-logloss:0.34048
[9]	train-logloss:0.31936	eval-logloss:0.33493
[10]	train-logloss:0.31443	eval-logloss:0.33162
[11]	train-logloss:0.30943	eval-logloss:0.33060
[12]	train-logloss:0.30474	eval-logloss:0.32771
[13]	train-logloss:0.30069	eval-logloss:0.32494
[14]	train-logloss:0.29698	eval-logloss:0.32200
[15]	train-logloss:0.29339	eval-logloss:0.31998
[16]	train-logloss:0.28977	eval-logloss:0.31737
[17]	train-logloss:0.28657	eval-logloss:0.31635
[18]	train-logloss:0.28361	eval-logloss:0.31490
[19]	train-logloss:0.28089	eval-logloss:0.31268
[20]	train-logloss:0.27861	eval-logloss:0.31122
[21]	train-logloss:0.27576	eval-logloss:0.31059
[22]	train-logloss:0.27320	eval-logloss:0.30932


## xgboost cross validation

In [27]:
data_dmatrix = xgb.DMatrix(data=X, label=y)

# パラメータ設定
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.1,
    'gamma': 0.0,
    'alpha': 0.0,
    'lambda': 1.0,
    'min_child_weight': 1,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

# 交差検証
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=42)

best_rounds = cv_results.shape[0]
final_model = xgb.train(params, data_dmatrix, num_boost_round=best_rounds)

# 結果の表示
print(cv_results)

    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.801896       0.006215       0.760270      0.027313
1         0.839041       0.004114       0.796919      0.012115
2         0.852312       0.002187       0.806383      0.014094
3         0.856661       0.001208       0.809960      0.012418
4         0.861130       0.002944       0.808063      0.011524
5         0.868050       0.002360       0.812093      0.007983
6         0.869706       0.001978       0.810761      0.009077
7         0.873276       0.002569       0.810492      0.007205
8         0.876097       0.003540       0.812704      0.007018
9         0.879355       0.003924       0.812738      0.007015
10        0.881741       0.003936       0.812371      0.005026
11        0.882648       0.003299       0.813114      0.004519
12        0.884396       0.003470       0.812964      0.005794
13        0.886618       0.002635       0.813223      0.004468
14        0.888513       0.003163       0.813538      0

In [28]:
X_submit = xgb.DMatrix(X_submit)

y_submit = final_model.predict(X_submit)

## lightgbm

In [38]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# パラメータの設定
params = {
    'boosting_type': 'gbdt',  # Gradient Boosting Decision Tree
    'objective': 'binary',  # 二値分類問題
    'metric': 'auc',  # 評価指標としてAUCを使用
    'learning_rate': 0.1,
    'num_leaves': 31,  # 葉の数（XGBoostのmax_depthと異なり、より大きい値）
    'min_child_weight': 1,
    'lambda_l1': 0.0,  # L1正則化
    'lambda_l2': 1.0,  # L2正則化
    'subsample': 0.8,  # データのサブサンプリング
    'colsample_bytree': 0.8,  # 特徴量のサブサンプリング
    'seed': 42
}

# モデルの訓練
model = lgb.train(
    params, 
    train_data, 
    num_boost_round=100, 
    valid_sets=[train_data, test_data]
)

# 予測
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# AUCスコアの計算
auc_score = roc_auc_score(y_test, y_pred)
print(f"AUC Score: {auc_score:.4f}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 398, number of negative: 2393
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.086169 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 410
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 32
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142601 -> initscore=-1.793851
[LightGBM] [Info] Start training from score -1.793851
AUC Score: 0.8245


## catboost

In [40]:
N_SPLITS = 5
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=0)
cv_list = list(skf.split(X, y))

In [41]:
def train_catboost(X, y, cv, params: dict = None):
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records, ))
    for i, (tr_idx, va_idx) in enumerate(cv):
        tr_x, va_x = X.iloc[tr_idx], X.iloc[va_idx]
        tr_y, va_y = y.iloc[tr_idx], y.iloc[va_idx]
        model = cb.CatBoostRegressor(**params)
        model.fit(tr_x, tr_y, eval_set=(va_x, va_y), use_best_model=True, verbose=100)
        oof_pred[va_idx] = model.predict(va_x)
        models.append(model)
    return oof_pred, models

In [42]:
params = {
    'loss_function': 'RMSE',
    'eval_metric': 'AUC',
    'iterations': 50,
    'learning_rate': 0.005,
    'depth': 6,
    'verbose': 200,
    'random_seed': 42,
}

oof, models = train_catboost(X, y, cv_list, params)

0:	test: 0.7626338	best: 0.7626338 (0)	total: 228ms	remaining: 11.2s
49:	test: 0.8049498	best: 0.8073746 (31)	total: 452ms	remaining: 0us

bestTest = 0.8073745819
bestIteration = 31

Shrink model to first 32 iterations.
0:	test: 0.7909030	best: 0.7909030 (0)	total: 6.19ms	remaining: 303ms
49:	test: 0.8343144	best: 0.8344314 (48)	total: 211ms	remaining: 0us

bestTest = 0.8344314381
bestIteration = 48

Shrink model to first 49 iterations.
0:	test: 0.7329556	best: 0.7329556 (0)	total: 4.89ms	remaining: 240ms
49:	test: 0.7784692	best: 0.7787390 (46)	total: 177ms	remaining: 0us

bestTest = 0.7787389757
bestIteration = 46

Shrink model to first 47 iterations.
0:	test: 0.7867321	best: 0.7867321 (0)	total: 4.53ms	remaining: 222ms
49:	test: 0.8207787	best: 0.8228782 (3)	total: 225ms	remaining: 0us

bestTest = 0.8228781977
bestIteration = 3

Shrink model to first 4 iterations.
0:	test: 0.7822455	best: 0.7822455 (0)	total: 6.78ms	remaining: 332ms
49:	test: 0.8119658	best: 0.8127428 (27)	total: 24

In [46]:
def predict_with_average(models, X_submit):
    # 各モデルによる予測値を格納する配列
    preds = np.zeros((len(X_submit), len(models)))
    
    # 各モデルを利用して予測を行う
    for i, model in enumerate(models):
        preds[:, i] = model.predict(X_submit)
    
    # 予測値の平均を求める
    final_preds = preds.mean(axis=1)
    return final_preds

# X_submitに対する予測値を計算
y_submit = predict_with_average(models, X_submit)

# Nural Network

# 提出

In [29]:
index = pd.read_csv('data/test.csv')['id'].values

df_submit = pd.DataFrame({
    "id": index,
    "prediction": y_submit
})

df_submit.head()

,id,prediction
0,3489,0.091986
1,3490,0.224697
2,3491,0.285730
3,3492,0.443214
4,3493,0.226650


In [33]:
path = 'submission/submit_6_xgboost_crossval.csv'

In [34]:
df_submit.to_csv(path, index=False, header=None)